In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/scores/scores.csv
/kaggle/input/dataset-fingerprints/167__M_Left_thumb_finger.jpg
/kaggle/input/dataset-fingerprints/189__F_Left_ring_finger.jpg
/kaggle/input/dataset-fingerprints/187__M_Right_index_finger.jpg
/kaggle/input/dataset-fingerprints/180__F_Right_middle_finger.jpg
/kaggle/input/dataset-fingerprints/181__M_Right_index_finger.jpg
/kaggle/input/dataset-fingerprints/186__M_Right_middle_finger.jpg
/kaggle/input/dataset-fingerprints/15__F_Left_thumb_finger.jpg
/kaggle/input/dataset-fingerprints/116__M_Right_ring_finger.jpg
/kaggle/input/dataset-fingerprints/158__M_Right_index_finger.jpg
/kaggle/input/dataset-fingerprints/108__M_Left_middle_finger.jpg
/kaggle/input/dataset-fingerprints/153__M_Right_thumb_finger.jpg
/kaggle/input/dataset-fingerprints/167__M_Right_ring_finger.jpg
/kaggle/input/dataset-fingerprints/179__M_Right_middle_finger.jpg
/kaggle/input/dataset-fingerprints/185__M_Right_little_finger.jpg
/kaggle/input/dataset-fingerprints/132__M_Left_ring_finger.jp

In [5]:
import os
import pandas as pd
import torch
import torchvision.transforms as transforms
from PIL import Image
import albumentations as A
from albumentations.pytorch import ToTensorV2
from albumentations import Compose, Resize, HorizontalFlip, VerticalFlip, Rotate, RandomBrightnessContrast, GaussianBlur, Normalize
import torchvision.models as models
import numpy as np
from torch.utils.data import DataLoader, Dataset
from torch import nn, optim
from torchvision.models import resnext50_32x4d
from torchvision.models import vgg16
from torch import nn
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
# Paths
image_folder = "/kaggle/input/dataset-fingerprints"  # Replace with your image folder path
csv_file = "/kaggle/input/scores/scores.csv"  # Replace with your CSV file path

# Load labels CSV
df = pd.read_csv(csv_file)
df.set_index("Image Name", inplace=True)  # Ensure column has image names

RuntimeError: Only a single TORCH_LIBRARY can be used to register the namespace prims; please put all of your definitions in a single TORCH_LIBRARY block.  If you were trying to specify implementations, consider using TORCH_LIBRARY_IMPL (which can be duplicated).  If you really intended to define operators for a single namespace in a distributed way, you can use TORCH_LIBRARY_FRAGMENT to explicitly indicate this.  Previous registration of TORCH_LIBRARY was registered at /dev/null:241; latest registration was registered at /dev/null:241

In [ ]:
# Augmentation pipeline with Resize
augmentation = Compose([
    Resize(224, 224),  # Resize all images to 224x224
    HorizontalFlip(p=0.5),  # Random horizontal flip
    VerticalFlip(p=0.5),  # Random vertical flip
    Rotate(limit=30, p=0.5),  # Random rotation (-30 to 30 degrees)
    Normalize(mean=[0.5], std=[0.5]),  # Normalize for grayscale images
    ToTensorV2()
])

# Function to preprocess and augment image
def preprocess_image(image_path):
    image = Image.open(image_path).convert("L")  # Convert to grayscale ('L' mode)
    image = augmentation(image=np.array(image))  # Apply augmentations and resizing
    return image
# Example Dataset Processing
processed_images = []
labels = []

for filename in os.listdir("/kaggle/input/dataset-fingerprints"):
        image_path = os.path.join("/kaggle/input/dataset-fingerprints", filename)
        try:
            processed_image = preprocess_image(image_path)
            label = df.loc[filename, "Score"]  # Fetch score from CSV
            
            processed_images.append(processed_image)
            labels.append(label)
            
            print(f"Processed: {filename}, Score: {label}")
        except Exception as e:
            print(f"Error processing {filename}: {e}")

# Convert to PyTorch Tensors for Training
if len(processed_images) > 0:
    X = torch.stack([img["image"] for img in processed_images])  # Images
    y = torch.tensor(labels, dtype=torch.float32)  # Labels
else:
    raise RuntimeError("No valid images were processed!")

print("Preprocessing complete! Ready for training.")

In [ ]:
# Define dataset class
class FingerprintDataset(Dataset):
    def __init__(self, csv_file, image_folder, transform=None):
        self.df = pd.read_csv(csv_file)
        self.image_folder = image_folder
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # Check if '.jpg' is already included in the filename
        filename = self.df.iloc[idx]["Image Name"]
        if not filename.endswith(".jpg"):
            filename += ".jpg"  # Append '.jpg' only if missing

        img_name = os.path.join(self.image_folder, filename)
        
        # Open the image file
        image = Image.open(img_name).convert("L")  # Convert to grayscale, use "L" if needed
        
        label = torch.tensor(self.df.iloc[idx]["Score"], dtype=torch.long)  # Classification (0-4)

        if self.transform:
            image = self.transform(image)
        
        return image, label

# Define transformations
import torchvision.transforms as transforms

# Define transformations
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  # Ensure the image is 1 channel (grayscale)
    transforms.Resize((224, 224)),  # Resize to consistent dimensions
    transforms.RandomHorizontalFlip(),  # Random horizontal flip
    transforms.RandomRotation(30),  # Random rotation
    transforms.ToTensor(),  # Convert to PyTorch Tensor
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize for grayscale input
])
dataset = FingerprintDataset(csv_file, image_folder, transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [ ]:
# Define the custom model
class ExtendedResNeXt(nn.Module):
    def __init__(self, num_classes=5):
        super(ExtendedResNeXt, self).__init__()
        self.base_model = resnext50_32x4d(pretrained=True)

        # Modify the first convolutional layer to accept grayscale input
        self.base_model.conv1 = nn.Conv2d(
            in_channels=1,  # Change input channels to 1 for grayscale
            out_channels=64,  # Keep output channels the same
            kernel_size=7,  # Original kernel size
            stride=2,  # Original stride
            padding=3,  # Original padding
            bias=False  # Original bias setting
        )

        # Remove the original classification head
        self.base_model.fc = nn.Identity()

        # Add custom layers after the base model
        self.fc_layers = nn.Sequential(
            nn.Linear(2048, 512),  # Input matches ResNeXt's output features
            nn.ReLU(),  # Activation function
            nn.Dropout(p=0.3),  # Dropout
            nn.Linear(512, 256),  # Second dense layer
            nn.ReLU(),  # Activation
            nn.Linear(256, 256),
            nn.Softmax(),
            nn.Linear(256, num_classes)# Final classification layer
        )
    
    def forward(self, x):
        x = self.base_model(x)  # Extract features from ResNeXt
        x = self.fc_layers(x)  # Pass through custom layers
        return x

# Instantiate the model
model = ExtendedResNeXt(num_classes=5)
model = model.to("cuda" if torch.cuda.is_available() else "cpu")
# Training loop
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Initialize lists to store loss and accuracy values
epoch_losses = []
epoch_accuracies = []

num_epochs = 100
all_labels = []  # Store true labels (only for the last epoch)
all_predictions = []  # Store predicted labels (only for the last epoch)

for epoch in range(num_epochs):
    total = 0  # Total number of examples
    correct = 0  # Total correct predictions
    running_loss = 0.0  # Track loss across batches

    for images, labels in dataloader:
        images, labels = images.to("cuda"), labels.to("cuda")

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimizer step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Accumulate loss for this epoch
        running_loss += loss.item()

        # Calculate accuracy
        _, predicted = torch.max(outputs, 1)  # Get max log-probability index
        total += labels.size(0)  # Count total labels
        correct += (predicted == labels).sum().item()  # Count correct predictions

        # Collect predictions and true labels only in the last epoch
        if epoch == num_epochs - 1:
            all_labels.extend(labels.cpu().numpy())  # Move to CPU for sklearn
            all_predictions.extend(predicted.cpu().numpy())  # Move to CPU for sklearn

    # Compute average loss and accuracy for this epoch
    epoch_loss = running_loss / len(dataloader)
    epoch_accuracy = 100 * correct / total

    epoch_losses.append(epoch_loss)
    epoch_accuracies.append(epoch_accuracy)

    # Print epoch loss and accuracy
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")

# Generate classification report (only after last epoch)
report = classification_report(all_labels, all_predictions, target_names=[f"Class {i}" for i in range(5)])
print("\nFinal Classification Report:\n", report)

In [ ]:
# Plot Loss and Accuracy Curves
plt.figure(figsize=(12, 5))

# Subplot for Loss
plt.subplot(1, 2, 1)
plt.plot(range(1, num_epochs + 1), epoch_losses, marker="o", label="Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Training Loss Over Epochs")
plt.legend()

# Subplot for Accuracy
plt.subplot(1, 2, 2)
plt.plot(range(1, num_epochs + 1), epoch_accuracies, marker="o", color="green", label="Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy (%)")
plt.title("Training Accuracy Over Epochs")
plt.legend()

plt.show()

In [ ]:
from torchvision.models import vgg16 
from torch import nn

class ExtendedVGG(nn.Module):
    def __init__(self, num_classes=5):  # Corrected __init__
        super(ExtendedVGG, self).__init__()
        
        # Load the VGG model and modify its first convolutional layer for grayscale input
        self.base_model = vgg16(pretrained=True)
        self.base_model.features[0] = nn.Conv2d(
            in_channels=1,  # Change input channels to 1 for grayscale
            out_channels=64,  # Keep output channels the same
            kernel_size=7,  # Original kernel size
            stride=3,  # Original stride
            padding=3,  # Original padding
            bias=False  # Original bias setting
        )
        
        # Remove the original classifier
        self.base_model.classifier = nn.Identity()

        # Add custom fully connected layers after the feature extractor
        self.fc_layers = nn.Sequential(
            nn.Linear(25088, 512),  # Input size matches the flattened feature map
            nn.ReLU(),  # Activation function
            nn.Dropout(p=0.3),  # Dropout for regularization
            nn.Linear(512, 256),  # Second dense layer
            nn.ReLU(),  # Activation
            nn.Linear(256, num_classes)
            # Final classification layer
        )

    def forward(self, x):
        x = self.base_model.features(x)  # Extract features with VGG
        x = x.view(x.size(0), -1)  # Flatten the feature map
        x = self.fc_layers(x)  # Pass through custom layers
        return x

In [ ]:
model = ExtendedVGG(num_classes=5)
model = model.to("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

import matplotlib.pyplot as plt
from sklearn.metrics import classification_report

# Initialize lists to store loss and accuracy values
epoch_losses = []
epoch_accuracies = []

num_epochs = 150
all_labels = []  # Store true labels (only for the last epoch)
all_predictions = []  # Store predicted labels (only for the last epoch)

for epoch in range(num_epochs):
    total = 0  # Total number of examples
    correct = 0  # Total correct predictions
    running_loss = 0.0  # Track loss across batches

    for images, labels in dataloader:
        images, labels = images.to("cuda"), labels.to("cuda")

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimizer step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Accumulate loss for this epoch
        running_loss += loss.item()

        # Calculate accuracy
        _, predicted = torch.max(outputs, 1)  # Get max log-probability index
        total += labels.size(0)  # Count total labels
        correct += (predicted == labels).sum().item()  # Count correct predictions

        # Collect predictions and true labels only in the last epoch
        if epoch == num_epochs - 1:
            all_labels.extend(labels.cpu().numpy())  # Move to CPU for sklearn
            all_predictions.extend(predicted.cpu().numpy())  # Move to CPU for sklearn

    # Compute average loss and accuracy for this epoch
    epoch_loss = running_loss / len(dataloader)
    epoch_accuracy = 100 * correct / total

    epoch_losses.append(epoch_loss)
    epoch_accuracies.append(epoch_accuracy)

    # Print epoch loss and accuracy
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")

# Generate classification report (only after last epoch)
report = classification_report(all_labels, all_predictions, target_names=[f"Class {i}" for i in range(5)])
print("\nFinal Classification Report:\n", report)

# Plot Loss and Accuracy Curves
plt.figure(figsize=(12, 5))

# Subplot for Loss
plt.subplot(1, 2, 1)
plt.plot(range(1, num_epochs + 1), epoch_losses, marker="o", label="Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Training Loss Over Epochs")
plt.legend()

# Subplot for Accuracy
plt.subplot(1, 2, 2)
plt.plot(range(1, num_epochs + 1), epoch_accuracies, marker="o", color="green", label="Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy (%)")
plt.title("Training Accuracy Over Epochs")
plt.legend()

plt.show()